In [1]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
%load_ext autoreload
%autoreload 2

In [2]:
# from importlib import reload
from helper import run
import op
import script

In [3]:
from nbx import nbx

### Exercise 1

Write the `op_hash160` function.

#### Make [this test](/edit/code-ch06/op.py) pass: `op.py:OpTest:test_op_hash160`

In [3]:
# Exercise 1

# reload(op)
run(op.OpTest("test_op_hash160"))

.
----------------------------------------------------------------------
Ran 1 test in 0.004s

OK


In [4]:
from script import Script

In [5]:
z = 0x7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d
sec = bytes.fromhex('04887387e452b8eacc4acfde10d9aaf7f6d9a0f975aabb10d006e4da568744d06c61de6d95231cd89026e286df3b6ae4a894a3378e393e93a0f45b666329a0ae34')
sig = bytes.fromhex('3045022000eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c022100c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab601')
script_pubkey = Script([sec, 0xac])
script_sig = Script([sig])
combined_script = script_sig + script_pubkey
print(combined_script.evaluate(z))

True


### Exercise 2

Write the `op_checksig` function in `op.py`

#### Make [this test](/edit/code-ch06/op.py) pass: `op.py:OpTest:test_op_checksig`

In [6]:
# Exercise 2

# reload(op)
run(op.OpTest("test_op_checksig"))

.
----------------------------------------------------------------------
Ran 1 test in 0.194s

OK


### Exercise 3

Create a ScriptSig that can unlock this ScriptPubKey. Note `OP_MUL` multiplies the top two elements of the stack.

`767695935687`

* `56 = OP_6`
* `76 = OP_DUP`
* `87 = OP_EQUAL`
* `93 = OP_ADD`
* `95 = OP_MUL`

In [8]:
# exploration /scratch work for Exercise 3

from script import Script

In [41]:
script_pubkey = Script([0x76, 0x76, 0x95, 0x93, 0x56, 0x87])

In [42]:
script_pubkey

OP_DUP OP_DUP OP_MUL OP_ADD OP_6 OP_EQUAL

In [ ]:
stack = []
x = 1
stack.append(x)          # op_x
stack.append(stack[-1])  # op_dup
stack.append(stack[-1])  # op_dup
mul1 = stack.pop() 
mul2 = stack.pop()
stack.append(mul1 * mul2)

assert stack[0] == 6



(x*x)+x = 6

x^2 + x - 6 = 0

-b +/- sqrt(b^2 - 4ac) / 2a

In [47]:
a = 1
b = 1
c = -6

x1 = (-b - ((b**2 - 4*a*c)**0.5)) / (2*a)
x2 = (-b + ((b**2 - 4*a*c)**0.5)) / (2*a)
x1, x2

(-3.0, 2.0)

In [91]:
from op import encode_num

In [93]:
0x52

82

In [ ]:
encode_num()

In [40]:
script_pubkey.evaluate(0)

hey


False

In [17]:
from logging import getLogger

In [19]:
log = getLogger("script")

In [21]:
getLogger("blah blah")

<Logger blah blah (WARNING)>

In [20]:
log.info

<bound method Logger.info of <Logger script (WARNING)>>

### Exercise 3

Create a ScriptSig that can unlock this ScriptPubKey. Note `OP_MUL` multiplies the top two elements of the stack.

`767695935687`

* `56 = OP_6`
* `76 = OP_DUP`
* `87 = OP_EQUAL`
* `93 = OP_ADD`
* `95 = OP_MUL`

##### The problem

In [51]:
script_pubkey = Script([0x76, 0x76, 0x95, 0x93, 0x56, 0x87])
script_sig = Script([82])  # FILL THIS IN
combined_script = script_sig + script_pubkey
print(combined_script.evaluate(0))

True


In [57]:
ser = script_pubkey.serialize()
ser

b'\x06vv\x95\x93V\x87'

In [58]:
raw_ser = script_pubkey.raw_serialize()
raw_ser

b'vv\x95\x93V\x87'

In [61]:
bytes.hex(raw_ser)  # bingo

'767695935687'

In [59]:
bytes.hex(ser)

'06767695935687'

### Exercise 4

Figure out what this Script is doing:

`6e879169a77ca787`

* `69 = OP_VERIFY`
* `6e = OP_2DUP`
* `7c = OP_SWAP`
* `87 = OP_EQUAL`
* `91 = OP_NOT`
* `a7 = OP_SHA1`

Use the `Script.parse` method and look up what various opcodes do at https://en.bitcoin.it/wiki/Script

In [76]:
# Exercise 4

from script import Script

script_pubkey = Script([0x6e, 0x87, 0x91, 0x69, 0xa7, 0x7c, 0xa7, 0x87])
script_sig = Script([81, 82])  # FILL THIS IN
combined_script = script_sig + script_pubkey
print(combined_script.evaluate(0))

False


In [80]:
print("\n".join(str(script_pubkey).split(" ")))

OP_2DUP
OP_EQUAL
OP_NOT
OP_VERIFY
OP_SHA1
OP_SWAP
OP_SHA1
OP_EQUAL


#### Describing the script
So this is basically a pinata for a sha1 collision. It will only return True if two different inputs have the same SHA1.

 - first it copies the top two elements on the stack
 - then it checks if those two elements are equal, if they are then script terminates in error code via `OP_VERIFY`
 - then it SHA1's each input. This will need remove the original input numbers from the stack and put the hash onto it.
 - One minor detail is using `OP_SWAP` to access the first input by putting it onto the top of the stack.

#### Understand the impact of changing op-byte-codes as it impacts the hex representation of the `raw_serialized` transaction.

In [65]:
bytes.hex(script_pubkey.raw_serialize())

'6e879169a77ca787'

In [83]:
first_script = Script([])  
combined_script = first_script + script_pubkey
bytes.hex(combined_script.raw_serialize())

'6e879169a77ca787'

In [84]:
first_script = Script([81])  
combined_script = first_script + script_pubkey
bytes.hex(combined_script.raw_serialize())

'516e879169a77ca787'

In [85]:
first_script = Script([81, 82])  
combined_script = first_script + script_pubkey
bytes.hex(combined_script.raw_serialize())

'51526e879169a77ca787'

##### Show how they don't always match

In [86]:
first_script = Script([])  
combined_script = first_script + script_pubkey
blank_hex = bytes.hex(combined_script.raw_serialize())

first_script = Script([81])  
combined_script = first_script + script_pubkey
op1_hex = bytes.hex(combined_script.raw_serialize())

first_script = Script([81, 82])  
combined_script = first_script + script_pubkey
op1op2_hex = bytes.hex(combined_script.raw_serialize())

In [88]:
blank_hex in op1_hex

True

In [89]:
blank_hex in op1op2_hex

True

In [90]:
op1_hex in op1op2_hex

False

#### Collisions

In [95]:
s1 = '255044462d312e330a25e2e3cfd30a0a0a312030206f626a0a3c3c2f576964746820\
32203020522f4865696768742033203020522f547970652034203020522f537562747970652035\
203020522f46696c7465722036203020522f436f6c6f7253706163652037203020522f4c656e67\
74682038203020522f42697473506572436f6d706f6e656e7420383e3e0a73747265616d0affd8\
fffe00245348412d3120697320646561642121212121852fec092339759c39b1a1c63c4c97e1ff\
fe017f46dc93a6b67e013b029aaa1db2560b45ca67d688c7f84b8c4c791fe02b3df614f86db169\
0901c56b45c1530afedfb76038e972722fe7ad728f0e4904e046c230570fe9d41398abe12ef5bc\
942be33542a4802d98b5d70f2a332ec37fac3514e74ddc0f2cc1a874cd0c78305a215664613097\
89606bd0bf3f98cda8044629a1'

In [96]:
s2 = '255044462d312e330a25e2e3cfd30a0a0a312030206f626a0a3c3c2f576964746820\
32203020522f4865696768742033203020522f547970652034203020522f537562747970652035\
203020522f46696c7465722036203020522f436f6c6f7253706163652037203020522f4c656e67\
74682038203020522f42697473506572436f6d706f6e656e7420383e3e0a73747265616d0affd8\
fffe00245348412d3120697320646561642121212121852fec092339759c39b1a1c63c4c97e1ff\
fe017346dc9166b67e118f029ab621b2560ff9ca67cca8c7f85ba84c79030c2b3de218f86db3a9\
0901d5df45c14f26fedfb3dc38e96ac22fe7bd728f0e45bce046d23c570feb141398bb552ef5a0\
a82be331fea48037b8b5d71f0e332edf93ac3500eb4ddc0decc1a864790c782c76215660dd3097\
91d06bd0af3f98cda4bc4629b1'

In [97]:
s1 == s2

False

In [98]:
for i,e in enumerate(s1):
    if s2[i] != e:
        print(f"index: {i} | s1: {e} | s2: {s2[i]}")

index: 385 | s1: f | s2: 3
index: 391 | s1: 3 | s2: 1
index: 392 | s1: a | s2: 6
index: 398 | s1: 0 | s2: 1
index: 400 | s1: 3 | s2: 8
index: 401 | s1: b | s2: f
index: 406 | s1: a | s2: b
index: 407 | s1: a | s2: 6
index: 408 | s1: 1 | s2: 2
index: 409 | s1: d | s2: 1
index: 415 | s1: b | s2: f
index: 416 | s1: 4 | s2: f
index: 417 | s1: 5 | s2: 9
index: 422 | s1: d | s2: c
index: 423 | s1: 6 | s2: c
index: 424 | s1: 8 | s2: a
index: 430 | s1: 4 | s2: 5
index: 432 | s1: 8 | s2: a
index: 433 | s1: c | s2: 8
index: 438 | s1: 1 | s2: 0
index: 439 | s1: f | s2: 3
index: 440 | s1: e | s2: 0
index: 441 | s1: 0 | s2: c
index: 446 | s1: f | s2: e
index: 447 | s1: 6 | s2: 2
index: 449 | s1: 4 | s2: 8
index: 455 | s1: 1 | s2: 3
index: 456 | s1: 6 | s2: a
index: 462 | s1: c | s2: d
index: 464 | s1: 6 | s2: d
index: 465 | s1: b | s2: f
index: 470 | s1: 5 | s2: 4
index: 471 | s1: 3 | s2: f
index: 472 | s1: 0 | s2: 2
index: 473 | s1: a | s2: 6
index: 479 | s1: 7 | s2: 3
index: 480 | s1: 6 | s2: d
i